# State Selection 

**State selection criteria:** 
1. Data available from 2000 - 2016
2. Data available for most counties from 2000 - 2016

**Selected States**
1. Michigan
2. Pennsylvania
3. Delaware
4. Nevada
5. Washington 

# Importing Data

In [1]:
import pandas as pd

In [2]:
tracts_df = pd.read_csv('./data/all_tracts.csv')

In [3]:
tracts_df

,GEOID,year,name,parent.location,population,poverty.rate,renter.occupied.households,pct.renter.occupied,median.gross.rent,median.household.income,...,pct.nh.pi,pct.multiple,pct.other,eviction.filings,evictions,eviction.rate,eviction.filing.rate,low.flag,imputed,subbed
0,2013000100,2009,1.00,"Aleutians East Borough, Alaska",2959.0,8.37,263.0,58.59,654.0,56250.0,...,0.34,5.58,0.14,0.0,NaN,NaN,0.00,0,0,1
1,2013000100,2010,1.00,"Aleutians East Borough, Alaska",3141.0,6.28,268.0,48.46,847.0,58125.0,...,0.60,3.72,0.03,0.0,NaN,NaN,0.00,0,0,1
2,2013000100,2013,1.00,"Aleutians East Borough, Alaska",3304.0,12.74,274.0,46.52,930.0,61518.0,...,1.12,3.18,0.00,0.0,NaN,NaN,0.00,0,0,1
3,2013000100,2015,1.00,"Aleutians East Borough, Alaska",3304.0,12.74,278.0,46.52,930.0,61518.0,...,1.12,3.18,0.00,0.0,NaN,NaN,0.00,0,0,1
4,2016000100,2016,1.00,"Aleutians West Census Area, Alaska",1065.0,12.43,128.0,41.67,934.0,57500.0,...,3.76,2.82,0.56,0.0,NaN,NaN,0.00,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250634,56043000301,2016,3.01,"Washakie County, Wyoming",2578.0,5.91,379.0,37.40,634.0,34643.0,...,0.00,2.83,0.00,2.0,2.0,0.53,0.53,0,0,0
250635,56043000302,2013,3.02,"Washakie County, Wyoming",2566.0,8.99,256.0,24.76,543.0,55192.0,...,0.00,4.21,0.00,1.0,1.0,0.39,0.39,0,0,0
250636,56043000302,2014,3.02,"Washakie County, Wyoming",2566.0,8.99,261.0,24.76,543.0,55192.0,...,0.00,4.21,0.00,2.0,2.0,0.77,0.77,0,0,0
250637,56043000302,2015,3.02,"Washakie County, Wyoming",2566.0,8.99,266.0,24.76,543.0,55192.0,...,0.00,4.21,0.00,0.0,0.0,0.00,0.00,0,0,0


In [4]:
tracts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250639 entries, 0 to 250638
Data columns (total 27 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   GEOID                       250639 non-null  int64  
 1   year                        250639 non-null  int64  
 2   name                        250639 non-null  float64
 3   parent.location             250639 non-null  object 
 4   population                  250607 non-null  float64
 5   poverty.rate                250607 non-null  float64
 6   renter.occupied.households  250609 non-null  float64
 7   pct.renter.occupied         250607 non-null  float64
 8   median.gross.rent           247914 non-null  float64
 9   median.household.income     249288 non-null  float64
 10  median.property.value       248169 non-null  float64
 11  rent.burden                 248476 non-null  float64
 12  pct.white                   250607 non-null  float64
 13  pct.af.am     

# Cleaning 

### Dropping States

Dropping due to missing years of data
1. ND
2. SD
3. AK
4. SC
5. NJ
6. MA
7. CT
8. RI
9. NH

In [5]:
drop_state_list = ['North Dakota', 'South Dakota', 'Arkansas', 
                   'South Carolina', 'New Jersey', 'Massachusetts',
                  'Connecticut', 'Rhode Island', 'New Hampshire']

for state in drop_state_list:
    drop_indices = tracts_df[tracts_df['parent.location'].str.contains(state)].index
    tracts_df.drop(drop_indices, inplace=True)

### Creating 'States' & 'County' Column 

In [6]:
tracts_df['state'] = tracts_df['parent.location'].str.rpartition(', ')[2]

tracts_df['county'] = tracts_df['parent.location'].str.rpartition(', ')[0]

tracts_df.drop('parent.location', axis=1, inplace=True)

1. 'county' may be an interesting feature for EDA, however it will create too many dummy variables during modeling 

### Differentiating Census Tracts by State

Census tracts are unique ONLY within each State, so I will add the name of the corresponding state to each census tract 

In [7]:
tracts_df['name'] = tracts_df['name'].astype(str)+'_'+tracts_df['state']

### Cleaning Null Target Variables

In [8]:
null_er_df = tracts_df[tracts_df['eviction.rate'].isna()]

In [9]:
null_er_df.shape

(2195, 28)

In [10]:
null_er_df[null_er_df['state'] == 'Alaska'].shape[0] == tracts_df[tracts_df['state'] == 'Alaska'].shape[0]

True

1. Alaska has no 'eviction.rate' data and must be dropped

In [17]:
tract_groupby = tracts_df.groupby(['name']).median()

In [18]:
tract_groupby

,GEOID,year,population,poverty.rate,renter.occupied.households,pct.renter.occupied,median.gross.rent,median.household.income,median.property.value,rent.burden,...,pct.nh.pi,pct.multiple,pct.other,eviction.filings,evictions,eviction.rate,eviction.filing.rate,low.flag,imputed,subbed
name,,,,,,,,,,,,,,,,,,,,,
1.01_California,6.105000e+09,2010.5,2747.5,15.930,300.0,22.77,872.0,33489.5,245000.085,45.45,...,0.085,2.535,0.05,6.5,6.5,2.165,2.165,0.0,0.0,0.0
1.01_Florida,1.208100e+10,2010.5,3222.0,10.325,900.5,61.64,913.0,31684.0,162600.000,43.55,...,0.000,1.790,0.09,30.0,12.0,1.165,3.155,0.0,0.0,0.0
1.01_Georgia,1.313900e+10,2003.0,3796.0,10.060,326.0,20.39,540.0,43861.0,96700.000,20.70,...,0.010,0.410,0.09,75.0,51.0,15.670,23.040,0.0,0.0,0.0
1.01_Iowa,1.915300e+10,2006.0,3194.0,24.430,844.0,68.21,641.0,27895.0,112800.000,31.10,...,0.000,3.600,0.00,102.0,31.0,3.450,10.520,0.0,0.0,0.0
1.01_Kentucky,2.106700e+10,2009.0,3415.5,25.455,1630.5,84.10,465.0,19149.0,138500.000,31.00,...,0.060,2.290,0.36,103.5,65.0,3.995,6.355,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997.05_New York,3.608110e+10,2015.0,2720.0,6.710,370.0,30.08,2256.0,71310.0,625300.000,14.60,...,0.000,1.840,0.00,NaN,NaN,NaN,NaN,0.0,0.0,0.0
998.01_New York,3.608110e+10,2015.0,7455.0,16.150,1985.0,72.14,1133.0,61434.0,385500.000,25.40,...,0.000,0.380,0.64,NaN,NaN,NaN,NaN,0.0,0.0,0.0
998.02_New York,3.608110e+10,2015.0,5391.0,12.100,2431.0,91.51,1003.0,34287.0,342800.000,33.40,...,0.000,0.820,0.00,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [ ]:
tract_groupby[tract_groupby['eviction.rate'].isna()]

1. 437 Census Tracts have no data for 'eviction.rate', so I will be dropping them 

In [ ]:
tract_groupby.reset_index(inplace=True)

In [ ]:
tract_groupby.name[0]

In [ ]:
drop_tract_list = []
for i in range(tract_groupby.shape[0]):
    drop_tract_list.append(tract_groupby.name[i])

In [ ]:
drop_tract_list